# Low-Level ReAct Implementation Demo

## Setup and Imports

To use this notebook: 
```bash 
pip install langchain python-dotenv langchain_community langchain_openai
```

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate
from langchain.utilities import WikipediaAPIWrapper

In [2]:
assert 'OPENAI_API_KEY' in os.environ, "Must set an OPENAI_API_KEY environment variable"

In [12]:
chat = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-1106")

## Create Mock Google Search Tool

In [13]:
tools = {}


def search_on_google(query: str):
    return f"Jason Derulo doesn't have a wife or partner."

tools["search_on_google"] = {
    "function": search_on_google,
    "description": "Searches on google for a query",
}

print("Mock Google Search Tool:", tools)

Mock Google Search Tool: {'search_on_google': {'function': <function search_on_google at 0x7c5ae949a160>, 'description': 'Searches on google for a query'}}


## Set Base Prompt

In [14]:
base_prompt = """
You will attempt to solve the problem of finding the answer to a question.
Use chain-of-thought reasoning to solve through the problem, using the
following pattern:

1. Observe the original question:
original_question: original_problem_text
2. Create an observation with the following pattern:
observation: observation_text
3. Create a thought based on the observation with the following pattern:
thought: thought_text
4. Use tools to act on the thought with the following pattern:
action: tool_name
action_input: tool_input

Do not guess or assume the tool results. Instead, provide a structured
output that includes the action and action_input.

You have access to the following tools: {tools}.

original_question: {question}
"""

print("Base Prompt:", base_prompt)

Base Prompt: 
You will attempt to solve the problem of finding the answer to a question.
Use chain-of-thought reasoning to solve through the problem, using the
following pattern:

1. Observe the original question:
original_question: original_problem_text
2. Create an observation with the following pattern:
observation: observation_text
3. Create a thought based on the observation with the following pattern:
thought: thought_text
4. Use tools to act on the thought with the following pattern:
action: tool_name
action_input: tool_input

Do not guess or assume the tool results. Instead, provide a structured
output that includes the action and action_input.

You have access to the following tools: {tools}.

original_question: {question}



## Initial Invocation

In [15]:
messages = SystemMessagePromptTemplate.from_template(template=base_prompt).format_messages(tools=tools, question="Is Jason Derulo with a partner?")

In [16]:
output = chat.invoke(messages)
print("Initial Output:", output.content)

Initial Output: observation: Jason Derulo may have a public relationship with a partner.
thought: I can search for recent news or social media posts about Jason Derulo to see if he is in a relationship.
action: search_on_google
action_input: Jason Derulo relationship status


In [17]:
for message in messages:
    print(message.content)


You will attempt to solve the problem of finding the answer to a question.
Use chain-of-thought reasoning to solve through the problem, using the
following pattern:

1. Observe the original question:
original_question: original_problem_text
2. Create an observation with the following pattern:
observation: observation_text
3. Create a thought based on the observation with the following pattern:
thought: thought_text
4. Use tools to act on the thought with the following pattern:
action: tool_name
action_input: tool_input

Do not guess or assume the tool results. Instead, provide a structured
output that includes the action and action_input.

You have access to the following tools: {'search_on_google': {'function': <function search_on_google at 0x7c5ae949a160>, 'description': 'Searches on google for a query'}}.

original_question: Is Jason Derulo with a partner?



In [18]:
output = chat.invoke(messages)
print(output.content)

observation: Jason Derulo may have a public relationship status that can be found through online sources.
thought: I can search online to find information about Jason Derulo's relationship status.
action: search_on_google
action_input: Jason Derulo relationship status


## Tool Invocation

In [19]:
import re

pattern = re.compile(r'action:\s*(?P<action>.*)\s+action_input:\s*(?P<action_input>.*)', re.DOTALL)
match = re.search(pattern, output.content)
assert match, "We expect a tool to be identified in this demo, chould check & handle in reality..."

print(match.groupdict())

{'action': 'search_on_google', 'action_input': 'Jason Derulo relationship status'}


In [20]:
tool_name = match["action"]
tool_input = match["action_input"]
print("Tool name:", tool_name)
print("Tool input:", tool_input)

Tool name: search_on_google
Tool input: Jason Derulo relationship status


In [21]:
tool_result = tools[tool_name]["function"](tool_input)

In [22]:

print(f"""The agent has opted to use the following tool:
tool_name: {tool_name}
tool_input: {tool_input}
tool_result: {tool_result}""")

The agent has opted to use the following tool:
tool_name: search_on_google
tool_input: Jason Derulo relationship status
tool_result: Jason Derulo doesn't have a wife or partner.


## Second Prompt

In [23]:
current_prompt = """
You are answering this query: Is Jason Derulo with a partner?

Based on the provided tool result:
tool_result: {tool_result}

Either provide the next observation, action, action_input, or the final
answer if available. If you are providing the final answer, you must return
the following pattern: "I've found the answer: final_answer"
"""

In [24]:
messages= SystemMessagePromptTemplate.from_template(template=current_prompt).format_messages(tool_result=tool_result)

for m in messages:
    print(m.content)


You are answering this query: Is Jason Derulo with a partner?

Based on the provided tool result:
tool_result: Jason Derulo doesn't have a wife or partner.

Either provide the next observation, action, action_input, or the final
answer if available. If you are providing the final answer, you must return
the following pattern: "I've found the answer: final_answer"



In [25]:
output = chat.invoke(messages)

print("Final Output:", output.content)

Final Output: Next observation: It seems that Jason Derulo is currently not in a relationship.


## Using Wikipedia Instead of Mock Google Search

Assumes
```bash
pip install wikipedia
```

In [26]:
# Create Wikipedia tool
wikipedia = WikipediaAPIWrapper()

tools["wikipedia_search"] = {
    "function": wikipedia.run,
    "description": "Searches Wikipedia for information about a topic",
}

print("Wikipedia Tool:", tools["wikipedia_search"])

Wikipedia Tool: {'function': <bound method WikipediaAPIWrapper.run of WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/opt/anaconda3/envs/cse434/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)>, 'description': 'Searches Wikipedia for information about a topic'}


In [28]:
from textwrap import fill
print(fill(wikipedia.run('What is the capital of France?')))

Page: Closed-ended question Summary: A closed-ended question is any
question for which a researcher provides research participants with
options from which to choose a response. Closed-ended questions are
sometimes phrased as a statement that requires a response. A closed-
ended question contrasts with an open-ended question, which cannot
easily be answered with specific information.    Page: Capital city
Summary: A capital city or just capital is the municipality holding
primary status in a country, state, province, department, or other
subnational division, usually as its seat of the government. A capital
is typically a city that physically encompasses the government's
offices and meeting places; the status as capital is often designated
by its law or constitution. In some jurisdictions, including several
countries, different branches of government are in different
settlements, sometimes meaning multiple official capitals. In some
cases, a distinction is made between the official (con

## Run ReAct with Wikipedia Tool

In [29]:
# Initial invocation with Wikipedia
output = chat.invoke((SystemMessagePromptTemplate
                      .from_template(template=base_prompt) 
                      .format_messages(tools=tools, 
                                       question="What is the capital of France and its population?")))
print("Initial Output (Wikipedia):", output.content)

Initial Output (Wikipedia): observation: The question is asking for two pieces of information: the capital of France and its population.

thought: I can first find the capital of France and then search for the population of that city.

action: wikipedia_search
action_input: France capital


In [30]:
# Tool invocation
match = re.search(pattern, output.content)
tool_name = match["action"]
tool_input = match["action_input"]
tool_result = tools[tool_name]["function"](tool_input)

from textwrap import fill
print(f"""The agent has opted to use the following tool:
tool_name: {tool_name}
tool_input: {tool_input}
tool_result: {fill(tool_result, subsequent_indent='        ')}""")

The agent has opted to use the following tool:
tool_name: wikipedia_search
tool_input: France capital
tool_result: Page: Capital punishment in France Summary: Capital punishment in
        France (French: peine de mort en France) is banned by Article
        66-1 of the Constitution of the French Republic, voted as a
        constitutional amendment by the Congress of the French
        Parliament on 19 February 2007 and simply stating "No one can
        be sentenced to the death penalty" (French: Nul ne peut être
        condamné à la peine de mort). The death penalty was already
        declared illegal on 9 October 1981 when President François
        Mitterrand signed a law prohibiting the judicial system from
        using it and commuting the sentences of the seven people on
        death row to life imprisonment. The last execution took place
        by guillotine, being the main legal method since the French
        Revolution; Hamida Djandoubi, a Tunisian citizen convicted of

In [31]:
# Second prompt
current_prompt = """
You are answering this query: What is the capital of France and its population?

Based on the provided tool result:
tool_result: {tool_result}

Either provide the next observation, action, action_input, or the final
answer if available. If you are providing the final answer, you must return
the following pattern: "I've found the answer: final_answer"
"""

output = chat.invoke((SystemMessagePromptTemplate.
                      from_template(template=current_prompt)
                      .format_messages(tool_result=tool_result)))

print("Final Output (Wikipedia):", output.content)

Final Output (Wikipedia): The capital of France is Paris. As for the population, the tool result does not provide the exact population of Paris, but it does mention that in 2021, there were 151,103 inhabitants in the metropolitan area of Cayenne, with 63,468 living in the city of Cayenne. This information is not relevant to the population of Paris. Therefore, I recommend conducting a specific search for the population of Paris.
